In [1]:
import numpy as np
import thermo
from environment import environmentThermo

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from skopt import BayesSearchCV
from skopt import gp_minimize


In [8]:
#begin with sampling training data (X, y) -> (weight fraction, thermophysical properties)

#brute force approach
batch_mass = 50
resolution = 1
num_dec = 1

compName = ['methanol', 'ethanol', 'butanol']

x = []
#x_1 = []
y = []

for i in range(0, int(batch_mass/resolution)+1):
  r = batch_mass - (i*resolution)
  for j in range(0, int(r/resolution)+1):
    runner = [i*resolution, j*resolution, r-j*resolution] #mass array
    x.append([x/np.sum(runner) for x in runner]) #gives array for mass fractions (x1, x2, ..., xN)
    #x_1.append(sum(runner))
    y.append(environmentThermo(runner, compName).thermophysicalProperties()) #gives thermophysical properties
    

In [11]:
print("X shape : {}".format(np.array(x).shape))
print("Y shape : {}".format(np.array(y).shape))

X shape : (1326, 3)
Y shape : (1326, 4)


In [12]:
#ensemble model will be working with 3 features and outputting 4 preds

In [14]:
#split data - for now, we can do 90/10, but we will also need a validation curve to prevent overfitting :)
#can also create k-fold CV
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [19]:
#get idea of the split datapoints
print("X_train shape : {}".format(np.array(X_train).shape))
print("Y_train shape : {}".format(np.array(y_train).shape))
print("X_test shape : {}".format(np.array(X_test).shape))
print("Y_test shape : {}".format(np.array(y_test).shape))

X_train shape : (1193, 3)
Y_train shape : (1193, 4)
X_test shape : (133, 3)
Y_test shape : (133, 4)


#### Experiment 1 : RF Regressor

In [15]:
from sklearn.ensemble import RandomForestRegressor

In [16]:
RF_model = RandomForestRegressor(n_estimators=100)

In [20]:
RF_model.fit(X_train, y_train)

RandomForestRegressor()

In [27]:
# Function to predict y from X using the ensemble model
def predict_y(X, model):
    return model.predict(X)

In [38]:
# Objective function to maximize
def FOM(X):
    y_pred = predict_y(X)
    fom_array = [0.2, -0.4, 0.2, 2]
    return -np.prod(np.power(y_pred[:4], fom_array[:4])) #made it negative to indicate max


In [41]:
# Perform Bayesian optimization
#i didn't define search space yet, will do it tomorrow
result = gp_minimize(FOM, n_calls=50)

TypeError: gp_minimize() missing 1 required positional argument: 'dimensions'